# Active Learning

In [1]:
import pandas as pd
import numpy as np
import uuid
import random

from sklearn.ensemble import GradientBoostingClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, matthews_corrcoef, confusion_matrix, classification_report

## Problem Statement

The business problem we are going to solve is associated to demographic modelling. Given some vital statistics associated to the users on our platform, we want to identify the gender associated to the user. We have a large amount of data but little to no labelled data. The goal is to build a model to identify the gender of a user.

## Generate Data

In case this needs to be said, if you're using this approach to solve a problem prominent to you, don't bother generating a dataset, use the one you're working with. I'm only generating a dataset for simplicity, reproducibility and to outline a use case on how such a problem could be solved using active learning.  

We are going to generate fake vital statistics associated to humans. The function we're building will randomly generate fake height, weight and age data. The resulting DataFrame from the function outlined below will yield a CSV with :   
- uuid (UUID4) : A unique identifier to the user  
- height (Integer) : The height of the user in cm  
- weight (Integer) : The weight of the user in pounds (lbs)  
- age (Integer) : The age of the user  
- gender (String) : The gender associated to the user if known  

In [2]:
def generate_data(n = 1000):
    '''
    This function will simulate the generation of gender data. It will randomly create
    columns associated to the height (cm), weight, age and gender. The gender column
    will not be completely filled out, it will have a few sample rows labelled to a gender.
    
    params:
        n (Integer) : The number of rows you want to synthesize
        
    returns:
        A dataframe with the columns of uuid, height, weight, age and gender.
            - uuid (UUID4) : A unique identifier to the user
            - height (Integer) : The height of the user in cm
            - weight (Integer) : The weight of the user in pounds (lbs)
            - age (Integer) : The age of the user
            - gender (String) : The gender associated to the user if known
        
    example:
        gender_df = generate_data(n = 1000)
    '''
    # we have more np.nan than Male or Female so that we can skew majority of the
    # data to be missing
    genders = ['Male', 'Female', np.nan, np.nan, np.nan, np.nan, np.nan, np.nan]
    height_range = (50, 200)
    weight_range = (30, 250)
    age_range = (3, 70)
    
    d = pd.DataFrame(
        {
            'uuid' : [uuid.uuid4() for _ in range(n)],
            'height' : [random.randint(height_range[0], height_range[1]) for _ in range(n)],
            'weight' : [random.randint(weight_range[0], weight_range[1]) for _ in range(n)],
            'age' : [random.randint(age_range[0], age_range[1]) for _ in range(n)],
            'gender' : [random.choice(genders) for _ in range(n)]
        }
    ).drop_duplicates()
    d = d.set_index('uuid')
    return d

In [3]:
gender_df = generate_data(n = 1000)
gender_df.shape

(1000, 4)

In [4]:
gender_df.head()

,height,weight,age,gender
uuid,,,,
c7f7bb27-5233-46f3-a699-da4c8fa8987b,133,118,5,NaN
43e56764-cf8a-420a-a35c-f91a71282da3,162,77,13,NaN
6e3f9de8-3de7-4e39-8969-344d500ff2f1,81,153,11,Female
b7758055-9b18-4be5-b073-fe33b997bae9,186,105,24,Male
a4283816-7538-4411-94ad-44429d0471bb,127,247,39,NaN


Of course, since we randomly generated data, we will have weird labelled values, like a 11 year old girl who weighs 153 pounds and is 81 centimeters tall. 

In [5]:
gender_df.describe()

,height,weight,age
count,1000.00000,1000.000000,1000.00000
mean,126.93600,141.352000,37.28200
std,43.79829,64.175935,19.44764
min,50.00000,30.000000,3.00000
25%,88.00000,87.000000,21.00000
50%,129.00000,145.000000,37.00000
75%,166.00000,197.000000,54.00000
max,200.00000,250.000000,70.00000


## Annotate Data

Here people would usually annotate their datasets associated to the labels, for the sake of speed and simplicity I'm going to select the randomly assigned gender data. There are various sampling stategies (like purposeful sampling and convenience sampling) you can investigate into to best select the data points to annotate. Purposeful sampling refers to selective sampling which would yield informative information to the model. Convenience sampling refers to easy to label information. Be aware, both of these methods does introduce a bias into the labelling process and could positively / negatively impact the model performance (hard to say and depends highly on the problem you're trying to solve) in comparison to random sampling.

In [6]:
gender_df['gender'].value_counts()

Male      128
Female    104
Name: gender, dtype: int64

In [7]:
train_df = gender_df[~gender_df['gender'].isna()].copy()
pred_df = gender_df[gender_df['gender'].isna()].copy()
print(train_df.shape, pred_df.shape)

(232, 4) (768, 4)


## Train Model

I'm going to use the Gradient Boosting model to train the binary classifier, in a real world situation you should try many different classifiers (like SVC, Logistic Regression, KNN, XG Boost, etc.) and evaluate the performance of each. You should also conduct hyperparameter tuning on these models to optimize the performance.

In [8]:
# feature, target breakdown
ft_cols = train_df.drop(columns = ['gender']).columns.tolist()
target_col = 'gender'

# train test split
x = train_df[ft_cols].values
y = train_df[target_col].values

x_train, x_test, y_train, y_test = train_test_split(
    x, 
    y,
    test_size = 0.3
)

In [9]:
# GBC classifier
clf = GradientBoostingClassifier()

# train the model
clf.fit(x_train, y_train)

GradientBoostingClassifier()

## Model Accuracy

In [10]:
def clf_eval(clf, x_test, y_test):
    '''
    This function will evaluate a sk-learn multi-class classification model based on its
    x_test and y_test values
    
    params:
        clf (Model) : The model you wish to evaluate the performance of
        x_test (Array) : Result of the train test split
        y_test (Array) : Result of the train test split
    
    returns:
        This function will return the following evaluation metrics:
            - Accuracy Score
            - Matthews Correlation Coefficient
            - Classification Report
            - Confusion Matrix
    
    example:
        clf_eval(
            clf,
            x_test,
            y_test
        )
    '''
    y_pred = clf.predict(x_test)
    y_true = y_test
    
    y_pred = clf.predict(x_test)
    test_acc = accuracy_score(y_test, y_pred)
    print("Testing Accuracy : ", test_acc)
    
    print("MCC Score : ", matthews_corrcoef(y_true, y_pred))
    
    print("Classification Report : ")
    print(classification_report(y_test, clf.predict(x_test)))
    
    
clf_eval(
    clf,
    x_test,
    y_test
)

Testing Accuracy :  0.5
MCC Score :  0.004973585797187662
Classification Report : 
              precision    recall  f1-score   support

      Female       0.42      0.52      0.46        29
        Male       0.59      0.49      0.53        41

    accuracy                           0.50        70
   macro avg       0.50      0.50      0.50        70
weighted avg       0.52      0.50      0.50        70



## Predict

In [11]:
pred_df['pred_proba'] = pred_df[ft_cols].apply(lambda x : dict(
    zip(clf.classes_, clf.predict_proba(x.values[None])[0])
), axis = 1)
pred_df['pred'] = pred_df[ft_cols].apply(lambda x : clf.predict(x.values[None])[0], axis = 1)

## Get Annotated & To Annoate Predictions

Predictions with low predicted probabilities will be annotated manually by the user, while predictions which have high predicted probability will be assumed to be a good prediction generated by the model. This will allow us to increase our labelled data and retrain a new model based on the best performing results of the previous model.

In [12]:
# low predictions with threshold <= 0.6
low_th = 0.6

# high predictions with threshold >= 0.9
high_th = 0.9

def check_preds(pred_dct, low_th = low_th, high_th = high_th):
    '''
    This function will check the dictionary associated to the prediction probabilities
    generated by the model. It will return either `annotate`, `annotated` or `neither`
    as a result.
    
    params:
        pred_dct (Dictionary) : The keys are the classes, values are the predicted proba
        low_th (Integer) : The low prediction proba threshold
        high_th (Integer): The high prediction proba threshold
        
    returns:
        This function will return the following:
            `annotate` : if the maximum value in the pred_dct is less than or equal to
                         the low_th
            `annotated` : if the maximum value in the pred_dct is greater than or equal to
                          the high_th
            'neither' : If it does not fall in the other two ranges
            
    example:
        pred_df['annotate_decision'] = pred_df['pred_proba'].apply(check_preds)
    '''
    
    max_val = max(list(pred_dct.values()))
    if max_val <= low_th:
        return 'annotate'
    elif max_val >= high_th:
        return 'annotated'
    else:
        return 'neither'

In [13]:
pred_df['annotate_decision'] = pred_df['pred_proba'].apply(check_preds)

In [14]:
pred_df['annotate_decision'].value_counts()

neither      504
annotate     144
annotated    120
Name: annotate_decision, dtype: int64

In [15]:
annotated_df = pred_df[pred_df['annotate_decision'] == 'annotated'].copy()
annotate_df = pred_df[pred_df['annotate_decision'] == 'annotate'].copy()

I'm just going to randomly assign a gender out of simplicity (since this is randomly generated data), but in reality you should manually label this.

In [16]:
gender_choices = ['Male', 'Female', np.nan, np.nan, np.nan]
annotate_df['gender'] = [random.choice(gender_choices) for _ in range(annotate_df.shape[0])]
annotate_df = annotate_df[~annotate_df['gender'].isna()].copy()

## Update Labelled Data

In [17]:
annotated_df['gender'] = annotated_df['pred']

In [18]:
annotated_df = pd.concat([annotated_df, annotate_df, train_df])

In [19]:
annotated_df.shape

(401, 7)

## Retrain Model

In [20]:
# feature, target breakdown
ft_cols = ['height', 'weight', 'age']
target_col = 'gender'

# train test split
x = annotated_df[ft_cols].values
y = annotated_df[target_col].values

x_train, x_test, y_train, y_test = train_test_split(
    x, 
    y,
    test_size = 0.3
)

In [21]:
# GBC classifier
clf = GradientBoostingClassifier()

# train the model
clf.fit(x_train, y_train)

GradientBoostingClassifier()

## Model Accuracy

In [22]:
clf_eval(
    clf,
    x_test,
    y_test
)

Testing Accuracy :  0.7520661157024794
MCC Score :  0.5242538184376094
Classification Report : 
              precision    recall  f1-score   support

      Female       0.85      0.68      0.76        69
        Male       0.67      0.85      0.75        52

    accuracy                           0.75       121
   macro avg       0.76      0.76      0.75       121
weighted avg       0.77      0.75      0.75       121



## Caveats

Be aware that this example above was associated to randomly generated data, this example was to show the power of active learning and how to use it to solve problems when there are a limited amount of labelled data. Although the performance of the model improved, the model is essentially learning randomness so do take these results with a grain of salt. 

---